In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils, experiment_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-11 22:15:19 __main__ INFO     torch.__version__='2.4.1+cu121', torch.version.cuda='12.1'
2024-10-11 22:15:19 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-10-11 22:15:19 __main__ INFO     transformers.__version__='4.44.2'


In [3]:
from src.models import ModelandTokenizer

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2024-10-11 22:16:01 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.02s/it]

2024-10-11 22:16:09 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-8B-Instruct> | size: 15316.516 MB | dtype: torch.float16 | device: cuda:0


In [29]:
prompt_template = """Assume that you are implementing a Caeser Cipher. You will be given a capital letter (A-Z) and a shift value (0-9). Return the letter that results from shifting the input letter by the input shift value. For example:
A#8->I
X#3->A
Z#8->H
{}#{}->"""

name = "J"
shift = 5

prompt = prompt_template.format(name, shift)

print(prompt)
print("=")

Assume that you are implementing a Caeser Cipher. You will be given a capital letter (A-Z) and a shift value (0-9). Return the letter that results from shifting the input letter by the input shift value. For example:
A#8->I
X#3->A
Z#8->H
J#5->
=


In [30]:
from src.tokens import prepare_input
from src.functional import predict_next_token

inputs = prepare_input(prompts=[prompt], tokenizer=mt)

predict_next_token(
    mt=mt, inputs=inputs
)

[[PredictedToken(token='F', prob=0.1361638456583023, logit=16.578125, token_id=37),
  PredictedToken(token='E', prob=0.1361638456583023, logit=16.578125, token_id=36),
  PredictedToken(token='G', prob=0.09655461460351944, logit=16.234375, token_id=38),
  PredictedToken(token='O', prob=0.0728832557797432, logit=15.953125, token_id=46),
  PredictedToken(token='K', prob=0.059485577046871185, logit=15.75, token_id=42)]]

In [27]:
for t_id in range(inputs.input_ids.shape[1]):
    print(mt.tokenizer.decode(inputs.input_ids[0, t_id].item()))

<|begin_of_text|>
Given
 the
 name
 of
 a
 person
 and
 a
 number
 (<
10
),
 give
 the
 first
 letter
 the
 name
 starts
 with
 if
 you
 encrypt
 it
 with
 a
 Caesar
 cipher
 with
 the
 given
 number
.

Alice
#
3
->
D


Bob
#
1
->
C


David
#
5
->
I


Jon
#
5
->
